In [1]:
import ast 
path = "/home/user/Documents/OLXScraper/olxprices/files"

def read_data(yy, mm, dd):
    if len(dd) == 1:
        dd = '0' + dd
    file_path = path + '/alugueisOLX_' + str(yy) + '-' + str(mm) + '-' + str(dd) + '.txt'
    data = []
    
    try:
        with open(file_path) as f:
            for line in f:
                data.append(ast.literal_eval(line.replace('\\u00e3', 'ã').replace('\\u00e9', 'é').replace('\\u00e2', 'â') \
                            .replace('\\u00e1', 'á').replace('\\u00ed', 'í').replace('\\u00f3', 'ó') \
                            .replace('\\u00b2', '2').replace('\\u00f4', 'ô').replace('\\u00ea', 'ê') \
                            .replace('\\u00e7', 'ç').replace('\\u00f5', 'õ').replace('\\u00fa', 'ú') \
                            .replace('\\u00aa', '°').replace('\\u00e0', 'à').replace('\\u00c1', 'Á') \
                            .replace('\\u00c9', 'É').replace('\\u00d3', 'Ó').replace('\\u00ca', 'Ê') \
                            .replace('\\u00ba', '°').replace('\\u00cd', 'Í').replace('\\u00b0', '°') \
                            .replace('\\u00fce','ü').replace('\\u00c2', 'Ã').replace('\\u00c7', 'Ç') \
                            .replace('\\"', '').replace('\\u00d4', 'Ô').replace('\\u00b4', '´') \
                            .replace('\\u00c3', 'Ã').replace('\n', '').replace('null', '"R$ 0"')))
    except IOError:
        print("File not accessible")
        return []
       
    return data

In [31]:
# Calcular similaridade:
import Levenshtein
import pandas as pd

goiania_neighborhood = pd.read_csv("/home/user/Documents/OLXScraper/olxprices/files/bairros_goiania.csv")
neighborhood_list = [i for i in goiania_neighborhood['Bairro']]

def best_neighborhood(string):
    best_distance = 100
    best_neighborhood = ''
    
    string = string.replace('Setor ', '')
    
    for neighborhood in neighborhood_list:
        distance = Levenshtein.distance(string, neighborhood)
        if distance == 1:
            return neighborhood
        elif distance < best_distance:
            best_distance = distance
            best_neighborhood = neighborhood
            
    return best_neighborhood if best_distance < 5 else None

Process:

In [4]:
vetor_processed = {}
complete_vector = []

for day in range(1, 31):
    vetor = read_data(2019,11,str(day))
    complete_vector.append([vetor, day])
    
    for line in vetor:
        
        #Calcular a data de retirada do anuncio:
        if line['id'] in vetor_processed:
            vetor_processed[line['id']][5] = day
        else:
            vetor_processed[line['id']] = [line['titulo'], line['regiao'].split(', '), line['detalhes'], line['preco'].replace(' ', '').split('R$')[1], \
                                           line['data_de_publicacao'] if line['data_de_publicacao'] != 'Hoje' and line['data_de_publicacao'] != 'Ontem' else str(day) + ' Nov', 1]

File not accessible
File not accessible
File not accessible


### Mapa que mostra valor médio dos imóveis por região

In [172]:
import pandas as pd
import plotly.express as px
import numpy as np

goiania_neighborhood = pd.read_csv("/home/user/Documents/OLXScraper/olxprices/files/bairros_goiania.csv")
neighborhood_list = { neighborhood : [0, 0, latitude, longitude] for neighborhood, latitude, longitude 
                     in zip(goiania_neighborhood['Bairro'], goiania_neighborhood['Latitude'], goiania_neighborhood['Longitude'])}
neighborhood_map = []

for line in vetor_processed:
    neighborhood = vetor_processed[line][1][1] if len(vetor_processed[line][1]) > 1 and vetor_processed[line][1][0] == "Goiânia" else None
    if neighborhood is not None:
        n = best_neighborhood(neighborhood)

        # Match com bairro e consta o valor do imovel e eh menor que 8000 reais:
        if n != None and 8000 > int(vetor_processed[line][3].replace('.', '')) > 100:
            neighborhood_list[n][0] +=1
            neighborhood_list[n][1] += int(vetor_processed[line][3].replace('.', ''))

            
for index in neighborhood_list:
    if neighborhood_list[index][0] != 0:
        neighborhood_map.append([index, 
                                 neighborhood_list[index][2],
                                 neighborhood_list[index][3],
                                 neighborhood_list[index][1]/neighborhood_list[index][0],
                                 neighborhood_list[index][0]])    

myData = np.array( neighborhood_map)
         
df=pd.DataFrame(data=myData,index=[i for i in range(len(myData))],columns=['Bairro','Latitude','Longitude','PrecoMedio','Size'])

df['Latitude'] = df['Latitude'].astype('float')
df['Longitude'] = df['Longitude'].astype('float')
df['PrecoMedio'] = df['PrecoMedio'].astype('float')
df['Size'] = df['Size'].astype('int')

fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_name="Bairro", color="PrecoMedio",
                        size="Size", color_discrete_sequence= px.colors.sequential.Plasma[-2::-1], zoom=11, height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Quantidade de imóveis por faixa de valor

#### Formato do vetor:
#### {'ID' : [ titulo, [cidade, bairro], descicao, preco, data_publicacao, tempo_no_ar] }

In [163]:
import plotly.graph_objects as go

precos_x = [i for i in range(0,2600,100)]
precos_y = [0 for i in range(len(precos_x))]

for line in vetor_processed:
    preco = float(vetor_processed[line][3].replace('.', ''))
    precos_y[len(precos_x) - 1 if preco > 2500 else int(preco // 100)]+= 1

fig = go.Figure([go.Bar(x=precos_x, y=precos_y)])
fig.show()

### Tempo médio que o anúncio ficou na plataforma:

In [164]:
import plotly.graph_objects as go

tempo_x = [i for i in range(1, 31)]
tempo_y = [0 for i in range(len(tempo_x))]

for line in vetor_processed:
    tempo = vetor_processed[line][5]
    tempo_y[tempo -1]+= 1

fig = go.Figure([go.Bar(x=tempo_x, y=tempo_y)])
fig.show()